In [2]:
%load_ext autoreload
%autoreload 2

from outlines import generate, models
from curverag import utils
from curverag.transformations import chunk_text
from curverag.graph import create_graph, KnowledgeGraph, Node, Edge, generate_prompt, create_graph_dataset
from curverag.utils import save_kg_dataset, split_triples, generate_to_skip, create_atth_dataset

import llama_cpp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Create dataset

In [3]:
max_tokens = 10000
n_ctx=10000

docs = [
    "The patient was diagnosed with type 2 diabetes mellitus and prescribed metformin 500mg twice daily.",
    "MRI scan revealed a small lesion in the left temporal lobe suggestive of low-grade glioma.",
    "Administer 5mg of lorazepam intravenously for acute seizure management.",
    "Blood tests showed elevated ALT and AST levels, indicating possible liver inflammation.",
    "The subject reported chronic lower back pain, managed with physical therapy and NSAIDs.",
    "CT angiography confirmed the presence of a pulmonary embolism in the right lower lobe.",
    "The patient underwent coronary artery bypass graft surgery without complications.",
    "Routine vaccination included MMR, tetanus, and influenza immunizations.",
    "Histopathology indicated ductal carcinoma in situ (DCIS) in the breast biopsy sample.",
    "The child presented with a persistent cough and fever, diagnosed as streptococcal pharyngitis."
]


# Load LLM

In [4]:
model = utils.load_model(
    llm_model_path="./models/Meta-Llama-3-8B-Instruct.Q6_K.gguf",
    tokenizer=llama_cpp.llama_tokenizer.LlamaHFTokenizer.from_pretrained("unsloth/Llama-3.2-1B-Instruct"),
    n_ctx=n_ctx,
    max_tokens=max_tokens
)

llama_model_load_from_file_impl: using device Metal (Apple M4 Pro) - 16383 MiB free
llama_model_loader: loaded meta data with 27 key-value pairs and 291 tensors from ./models/Meta-Llama-3-8B-Instruct.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3
llama_model_loader: - kv   5:                               general.tags arr[str,6]       = ["facebook", "meta", "pytorch", "llam...
llama_model_loader: - kv   6:            

# Create and save graph

In [5]:
graph = create_graph(model, docs, chunk_size=3000, max_tokens=max_tokens)

  0%|                                                                             | 0/10 [00:00<?, ?it/s]/Users/nathan/Documents/projects/curve_rag/.venv/lib/python3.10/site-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


"<|begin_of_text|><|start_header_id|>system<|end_header_id|>
       You are a world class AI model who extracts nodes and entities from documents to add to an exiting Knowledge Graph creation task. Put yur reply in JSON<|eot_id|>
        <|start_header_id|>user<|end_header_id|>
        Here's the json schema you must adhere to: <schema>{'$defs': {'Edge': {'properties': {'source': {'description': 'ID of the source node edge', 'title': 'Source', 'type': 'integer'}, 'target': {'description': 'ID of the target node edge', 'title': 'Target', 'type': 'integer'}, 'name': {'description': 'Name of the relationship for the edge. Maximum of 20 characters.', 'title': 'Name', 'type': 'string'}, 'is_directed': {'description': 'If true its a directed edge', 'title': 'Is Directed', 'type': 'boolean'}, 'description': {'description': 'A short description of the edge. Maximum of 50 characters.', 'title': 'Description', 'type': 'string'}}, 'required': ['source', 'target', 'name', 'is_directed', 'descripti

llama_perf_context_print:        load time =    5220.97 ms
llama_perf_context_print: prompt eval time =    5219.89 ms /   613 tokens (    8.52 ms per token,   117.44 tokens per second)
llama_perf_context_print:        eval time =   24346.94 ms /   521 runs   (   46.73 ms per token,    21.40 tokens per second)
llama_perf_context_print:       total time =   32783.23 ms /  1134 tokens
 10%|██████▉                                                              | 1/10 [00:32<04:55, 32.79s/it]

nodes=[Node(id=1, name='Patient', description='A person who is receiving medical treatment', alias=['individual', 'person']), Node(id=2, name='Diabetes Mellitus', description='A group of metabolic disorders characterized by high blood sugar levels', alias=['diabetes', 'DM']), Node(id=3, name='Metformin', description='A medication used to treat type 2 diabetes', alias=['glucophage', 'metformin hydrochloride']), Node(id=4, name='Type 2 Diabetes', description='A type of diabetes that is caused by insulin resistance and impaired insulin secretion', alias=['T2D', 'non-insulin-dependent diabetes']), Node(id=5, name='Daily', description='A unit of time', alias=['day', '24 hours']), Node(id=6, name='500mg', description='A dosage of medication', alias=['half a gram', '0.5 grams']), Node(id=7, name='Twice', description='A frequency of medication administration', alias=['every 12 hours', 'b.i.d.'])] edges=[Edge(source=1, target=2, name='has_disease', is_directed=True, description='The patient has

llama_perf_context_print:        load time =    5220.97 ms
llama_perf_context_print: prompt eval time =    7152.21 ms /  1005 tokens (    7.12 ms per token,   140.52 tokens per second)
llama_perf_context_print:        eval time =   19420.87 ms /   374 runs   (   51.93 ms per token,    19.26 tokens per second)
llama_perf_context_print:       total time =   29628.57 ms /  1379 tokens
 20%|█████████████▊                                                       | 2/10 [01:02<04:07, 30.93s/it]

...

nodes=[Node(id=31, name='Routine vaccination', description='A regular vaccination schedule', alias=['routine vaccination', 'immunization']), Node(id=33, name='MMR', description='A type of vaccine', alias=['measles, mumps, rubella', 'measles, mumps, rubella vaccine']), Node(id=34, name='Tetanus', description='A type of vaccine', alias=['tetanus vaccine', 'tetanus toxoid']), Node(id=35, name='Influenza', description='A type of vaccine', alias=['influenza vaccine', 'flu vaccine'])] edges=[Edge(source=31, target=33, name='includes', is_directed=True, description='The routine vaccination includes the MMR vaccine'), Edge(source=31, target=34, name='includes', is_directed=True, description='The routine vaccination includes the tetanus vaccine'), Edge(source=31, target=35, name='includes', is_directed=True, description='The routine vaccination includes the influenza vaccine')]
"<|begin_of_text|><|start_header_id|>system<|end_header_id|>
       You are a world class AI model who extracts nodes 

llama_perf_context_print:        load time =    5220.97 ms
llama_perf_context_print: prompt eval time =   23472.20 ms /  2635 tokens (    8.91 ms per token,   112.26 tokens per second)
llama_perf_context_print:        eval time =   16867.76 ms /   299 runs   (   56.41 ms per token,    17.73 tokens per second)
llama_perf_context_print:       total time =   45167.25 ms /  2934 tokens
 90%|███████████████████████████████████████████████████████████▍      | 9/10 [2:20:05<11:12, 672.76s/it]

nodes=[Node(id=36, name='Histopathology', description='The study of the microscopic structure of tissues', alias=['histopathology', 'microscopic examination']), Node(id=37, name='Ductal Carcinoma In Situ', description='A type of breast cancer', alias=['DCIS', 'ductal carcinoma in situ', 'breast cancer']), Node(id=38, name='Breast', description='The organ responsible for milk production in females', alias=['breast', 'mammary gland']), Node(id=39, name='Biopsy', description='A medical procedure to remove a sample of tissue for examination', alias=['biopsy', 'tissue sample'])] edges=[Edge(source=36, target=37, name='indicates', is_directed=True, description='The histopathology indicates the presence of DCIS'), Edge(source=37, target=38, name='located_in', is_directed=True, description='The DCIS is located in the breast'), Edge(source=38, target=39, name='sample_of', is_directed=True, description='The breast is a sample of tissue for examination')]
"<|begin_of_text|><|start_header_id|>syst

llama_perf_context_print:        load time =    5220.97 ms
llama_perf_context_print: prompt eval time =   24945.40 ms /  2866 tokens (    8.70 ms per token,   114.89 tokens per second)
llama_perf_context_print:        eval time =   26930.68 ms /   466 runs   (   57.79 ms per token,    17.30 tokens per second)
llama_perf_context_print:       total time =   60078.26 ms /  3332 tokens
100%|█████████████████████████████████████████████████████████████████| 10/10 [2:21:06<00:00, 846.60s/it]

nodes=[Node(id=40, name='Child', description='A young human being', alias=['young person', 'juvenile']), Node(id=41, name='Cough', description='A sudden and involuntary expulsion of air from the lungs', alias=['coughing', 'coughs']), Node(id=42, name='Fever', description='An elevated body temperature', alias=['feverish', 'febrile']), Node(id=43, name='Streptococcal Pharyngitis', description='An infection of the pharynx caused by streptococcal bacteria', alias=['strep throat', 'pharyngitis']), Node(id=44, name='Pharynx', description='The part of the throat that connects the nose and mouth', alias=['throat', 'pharynx']), Node(id=45, name='Streptococcus', description='A type of bacteria', alias=['strep', 'streptococcus'])] edges=[Edge(source=40, target=41, name='experiences', is_directed=True, description='The child experiences a cough'), Edge(source=40, target=42, name='experiences', is_directed=True, description='The child experiences a fever'), Edge(source=40, target=43, name='has_cond

In [6]:
graph.nodes[:5]

[Node(id=1, name='Patient', description='A person who is receiving medical treatment', alias=['individual', 'person']),
 Node(id=2, name='Diabetes Mellitus', description='A group of metabolic disorders characterized by high blood sugar levels', alias=['diabetes', 'DM']),
 Node(id=3, name='Metformin', description='A medication used to treat type 2 diabetes', alias=['glucophage', 'metformin hydrochloride']),
 Node(id=4, name='Type 2 Diabetes', description='A type of diabetes that is caused by insulin resistance and impaired insulin secretion', alias=['T2D', 'non-insulin-dependent diabetes']),
 Node(id=5, name='Daily', description='A unit of time', alias=['day', '24 hours']),
 Node(id=6, name='500mg', description='A dosage of medication', alias=['half a gram', '0.5 grams']),
 Node(id=7, name='Twice', description='A frequency of medication administration', alias=['every 12 hours', 'b.i.d.']),
 Node(id=8, name='MRI', description='A medical imaging technique', alias=['magnetic resonance imag

In [7]:
graph.edges[:5]

[Edge(source=1, target=2, name='has_disease', is_directed=True, description='The patient has the disease'),
 Edge(source=1, target=3, name='takes_medication', is_directed=True, description='The patient is taking the medication'),
 Edge(source=3, target=4, name='treats', is_directed=True, description='The medication treats the disease'),
 Edge(source=1, target=5, name='takes_medication_at', is_directed=True, description='The patient takes the medication at a specific time'),
 Edge(source=3, target=6, name='has_dosage', is_directed=True, description='The medication has a specific dosage'),
 Edge(source=1, target=7, name='takes_medication_frequency', is_directed=True, description='The patient takes the medication at a specific frequency'),
 Edge(source=1, target=8, name='uses', is_directed=True, description='The MRI uses a specific technique'),
 Edge(source=8, target=9, name='reveals', is_directed=True, description='The MRI reveals a lesion'),
 Edge(source=9, target=10, name='located_in',

In [10]:
create_graph_dataset(graph, dataset_name="medical_docs")